In [58]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [88]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class' : 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'})
        price = price.find("span", attrs={'class':'a-offscreen'}).string.strip()
    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
#         try:
#             rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
#         except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [89]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = 'https://www.amazon.com/s?k=playstation+5&sprefix=playstatrion%2Caps%2C261&ref=nb_sb_ss_ts-doa-p_1_12'

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[],"link":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['link'].append("https://www.amazon.com" + link)

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [94]:
amazon_df

,title,price,rating,reviews,availability,link
0,PlayStation 5 Console (PS5),$499.00,4.8 out of 5 stars,"14,102 ratings",In Stock,https://www.amazon.com/PlayStation-5-Console-C...
1,PlayStation 5 Console CFI-1102A,,4.9 out of 5 stars,"7,334 ratings",,https://www.amazon.com/PlayStation-5-Console/d...
2,PlayStation 5 Digital Edition,,4.9 out of 5 stars,"7,198 ratings",,https://www.amazon.com/PlayStation-5-Digital/d...
3,PlayStation 5 Pulse 3D Wireless Headset,$98.99,4.7 out of 5 stars,"19,723 ratings",Not Available,https://www.amazon.com/PlayStation-Pulse-3D-Wi...
4,PlayStation DualSense Wireless Controller - St...,$74.99,4.8 out of 5 stars,"74,863 ratings",Only 6 left in stock - order soon,https://www.amazon.com/PlayStation-DualSense-W...
5,WD_BLACK 2TB SN850 NVMe SSD for PS5 Consoles S...,$179.99,4.8 out of 5 stars,"3,079 ratings",In Stock,https://www.amazon.com/WD_BLACK-SN850-Consoles...
6,Saga of Sins - PlayStation 5,$39.99,,,"This item will be released on November 3, 2023.",https://www.amazon.com/Saga-Sins-PlayStation-5...
7,$50 PlayStation Store Gift Card [Digital Code],$50.00,4.7 out of 5 stars,"261,343 ratings",Available now,https://www.amazon.com/PlayStation-Store-Gift-...
8,Xbox Series X – Forza Horizon 5 Bundle,,4.8 out of 5 stars,"21,798 ratings",Only 1 left in stock - order soon,https://www.amazon.com/Xbox-X-Forza-Horizon-5-...
9,Human: Fall Flat - Dream Collection - PlayStat...,$29.99,,,"This item will be released on August 18, 2023.",https://www.amazon.com/Human-Fall-Flat-Dream-C...
